## Import libraries

In [375]:
import pandas as pd
import numpy as np 

In [358]:
input_file = "/Users/aoukachamoussa/Desktop/hackathon /data/bad_crop_targets.csv"
output_file = "cleaned_data.csv"

with open(input_file, "r", encoding="utf-8") as infile:
    lines = infile.readlines()

with open(output_file, "w", encoding="utf-8", newline="") as outfile:
    outfile.write('"SAMPLEID","LATITUDE","LONGITUDE","NDVI","PRIMARY_CROP_CLASSIFICATION"\n')
    
    for line in lines[1:]: 
        parts = line.strip().split(",")
        
        if len(parts) < 5:
            continue  
        
        lat = parts[-4]
        lon = parts[-3]
        ndvi = parts[-2]
        crop = parts[-1]
        sampleid_parts = parts[:-4]
        sampleid_clean = ''.join(sampleid_parts).replace(",", "")


        outfile.write(f'"{sampleid_clean}",{lat},{lon},{ndvi},"{crop}"\n')


In [359]:
data = pd.read_csv('cleaned_data.csv')

In [360]:
data.head(10)

,SAMPLEID,LATITUDE,LONGITUDE,NDVI,PRIMARY_CROP_CLASSIFICATION
0,3�]��<{��cN\u0018\u0000\n\t6,18.918968,121.880359,0.319048,MAIZE
1,3�]�\u001ac|\u0007�cN\u0018\u0000\noU,18.903895,121.868401,0.371333,MAIZE
2,3�]���|\u0019�cN\u0018\u0000\n�\u0000,18.902367,121.869741,0.359111,MAIZE
3,3�]���|\u0015�cN\u0018\u0000\n�$,18.918964,121.879789,0.379376,MAIZE
4,3�]���|\u0015�cN\u0018\u0000\n�$,18.912134,121.871099,0.364602,MAIZE
5,3�]���|\u0015�cN\u0018\u0000\n�$,18.904080,121.869065,0.367412,MAIZE
6,3�]��\n{��cN\u0018\u0000\n�*,18.923650,121.877574,0.384270,MAIZE
7,3�]��\u0013{��cN\u0018\u0000\n�*,18.919237,121.879977,0.370821,MAIZE
8,3�]�L\u0000|\u000f�cN\u0018\u0000\n��,18.903989,121.869065,0.295014,MAIZE
9,3�]��y{��cN\u0018\u0000\n*�,18.923560,121.877670,0.344326,MAIZE


## MOHAMMED DATA VIZ

In [361]:
# import pandas as pd
# import folium
# from folium import CircleMarker
# import matplotlib.cm as cm
# import matplotlib.colors as colors

# # Sample data (assuming you already have this in a CSV or DataFrame)
# data = data.dropna(subset=['LATITUDE', 'LONGITUDE', 'NDVI'])

# # Center map on the average coordinates
# map_center = [data['LATITUDE'].mean(), data['LONGITUDE'].mean()]
# m = folium.Map(location=map_center, zoom_start=14)

# df = pd.DataFrame(data)

# # Set center of the map to the average location
# map_center = [df['LATITUDE'].mean(), df['LONGITUDE'].mean()]
# m = folium.Map(location=map_center, zoom_start=14)

# # Normalize NDVI for color scaling
# norm = colors.Normalize(vmin=df['NDVI'].min(), vmax=df['NDVI'].max())
# colormap = cm.ScalarMappable(norm=norm, cmap='YlGn')

# # Plot each point
# for idx, row in df.iterrows():
#     folium.CircleMarker(
#         location=[row['LATITUDE'], row['LONGITUDE']],
#         radius=6,
#         color='black',
#         fill=True,
#         fill_opacity=0.7,
#         fill_color=colors.to_hex(colormap.to_rgba(row['NDVI'])),
#         popup=f"NDVI: {row['NDVI']:.3f} | Crop: {row['PRIMARY_CROP_CLASSIFICATION']}"
#     ).add_to(m)

# # Display the map (in Jupyter) or save to HTML
# m.save("map_visualization.html")


In [362]:
drone_base_data = pd.read_csv('/Users/aoukachamoussa/Desktop/hackathon /data/drone_bases.csv')
drone_base_data.head()

,CITY,LAT,LNG
0,Labbi Camp,16.5000,121.0667
1,Villa Remedios,16.7043,121.8306
2,Concong,17.6127,121.8016
3,Upas,18.0880,120.6098
4,Silauan Sur,16.7033,121.6778


## CLASSES DEFINITION

### normal hours check

In [363]:
import datetime

# to check if we're in the normal work hours, which are from 9am to 5pm

# def is_normal_work_hours():
#     current_time = datetime.datetime.now().time()
#     start_time = datetime.time(9, 0)  # 9:00 AM
#     end_time = datetime.time(17, 0)   # 5:00 PM
#     return start_time <= current_time <= end_time

def is_normal_work_hours(test_time):
    current_time = test_time
    start_time = datetime.time(9, 0)  # 9:00 AM
    end_time = datetime.time(17, 0)   # 5:00 PM
    return start_time <= current_time <= end_time

global normal_work_hours
normal_work_hours = is_normal_work_hours(datetime.datetime.now().time())

In [364]:
class drone:
    def __init__(self, drone_id, home_drone, target_field, max_time_flight, total_time_target, current_location, list_tickets, status = 'AVAILABLE', drone_model='JOUVAC CW-30E', speed=90):
        self.drone_id = drone_id
        self.drone_model = drone_model
        self.home_drone = home_drone
        self.target_field = target_field
        self.max_time_flight = max_time_flight
        self.total_time_target = total_time_target
        self.current_location = current_location
        self.list_tickets = list_tickets
        self.status = status
        self.speed = speed
        self.available_time_flight = max_time_flight

    def return_to_home(self):
        self.target_field = self.home_drone
        self.current_location = self.home_drone
        self.status = 'AV'

    def time_left(self):
        if self.total_time_target == None:
            return self.max_time_flight
        
        return self.max_time_flight - self.total_time_target
    

    

In [365]:
class drone_base:
    def __init__(self, base_id, position, capacity, city_name):
        self.base_id = base_id
        self.position = position
        self.capacity = capacity
        self.city_name = city_name
        self.list_drone = []

    def assign_drone(self, drone):
        if len(self.list_drone) < self.capacity:
            self.list_drone.append(drone)
            drone.home_drone = self.position
            drone.current_location = self.position
            drone.status = 'AVAILABLE'
            return 'Y'
        else:
            print("Drone base is at full capacity. Cannot assign more drones.")
            return 'N'
            
    def add_drone(self):
            pass

In [366]:
class ticket:
    def __init__(self, id, target_field, nvdi, drone_id=None, mission='examine', status='open'):
        self.id = id
        self.target_field = target_field
        self.nvdi = nvdi
        self.drone_id = drone_id
        self.mission = mission
        self.status = status

    def assign_drone(self, drone_id: drone):
        if drone_id.status == 'available':
            self.drone_id = drone_id
            drone_id.status = 'busy'
        else:
            print(f"Drone {drone_id} is not available for assignment.")

    def update_status(self, new_status: str):
        self.status = new_status
        if new_status == 'completed':
            self.drone_id.status = 'available'
        elif new_status == 'cancelled':
            self.drone_id.status = 'available'
        elif new_status == 'in_progress':
            self.drone_id.status = 'busy'


def create_all_tickets(crop_data):
    tickets = []
    i = 0
    for _ , row in crop_data.iterrows():
        ticket_id = i
        target_field = (row['LATITUDE'], row['LONGITUDE'])
        nvdi = row["NDVI"]
        mission = "EXAMINATION"
        ticket_instance = ticket(ticket_id, target_field, nvdi, mission=mission)
        tickets.append(ticket_instance)
        i+=1
    return tickets

def create_all_bases(base_data):
    bases = []
    i = 0
    for _ , row in base_data.iterrows():
        base_id = i
        capacity = 2
        city_name = row['CITY']
        position = (row['LAT'], row['LNG'])
        #list_drone = []
        base_instance = drone_base(base_id, position=position, capacity=capacity, city_name=city_name)
        bases.append(base_instance)
        i+=1
    return bases


## SCENARIO 1 : 100 TICKETS + 10 BASES + 2 DRONES/BASE

In [367]:
#load data
drone_base_df = pd.read_csv('/Users/aoukachamoussa/Desktop/hackathon /data/drone_bases.csv')

In [368]:
#GENERATE TICKETS
crop_data = data[:100]
tickets = create_all_tickets(crop_data)

#GENERATE DRONE BASES
drone_bases_test = create_all_bases(drone_base_df[:10])

In [369]:
#Printing generated tickets
print('************ Generating Tickets ************\n')
for _ in tickets:
    print(_.id, _.target_field, _.nvdi, _.mission, _.drone_id)

************ Generating Tickets ************

0 (18.918967716491903, 121.88035887111538) 0.319047619047619 EXAMINATION None
1 (18.90389484630481, 121.86840142250212) 0.3713327745180217 EXAMINATION None
2 (18.90236681513191, 121.86974088442432) 0.3591106814886418 EXAMINATION None
3 (18.91896428007593, 121.87978918395828) 0.3793755912961211 EXAMINATION None
4 (18.912134366273968, 121.8710994860073) 0.3646017699115044 EXAMINATION None
5 (18.90407961313326, 121.86906478097234) 0.3674121405750798 EXAMINATION None
6 (18.9236498208814, 121.8775740164441) 0.3842696629213483 EXAMINATION None
7 (18.919236506600832, 121.87997727405433) 0.3708206686930091 EXAMINATION None
8 (18.90398925304056, 121.86906538818512) 0.2950138504155125 EXAMINATION None
9 (18.923560034837273, 121.87766957013343) 0.3443258971871969 EXAMINATION None
10 (18.91914958218428, 121.88054756373754) 0.3871836295099623 EXAMINATION None
11 (18.918419826525277, 121.8794130055508) 0.3887530562347188 EXAMINATION None
12 (18.879094700

In [370]:
#Printing generated drone_bases
print('************ Generating Drone Bases ************\n')

for _ in drone_bases_test:
    print(_.base_id, _.position, _.capacity, _.city_name)

************ Generating Drone Bases ************

0 (16.5, 121.0667) 2 Labbi Camp
1 (16.7043, 121.8306) 2 Villa Remedios
2 (17.6127, 121.8016) 2 Concong
3 (18.088, 120.6098) 2 Upas
4 (16.7033, 121.6778) 2 Silauan Sur
5 (18.2626, 121.9327) 2 Casitan
6 (17.6103, 120.6759) 2 Bagaong
7 (16.972, 121.7091) 2 Concepcion
8 (18.3538, 121.4435) 2 Gubu
9 (17.1148, 121.7607) 2 Rizal


In [371]:
def get_drone_base_by_id(base_id):
    for base in drone_bases_test:
        if base.base_id == base_id:
            return base
    return None

In [372]:
drones_list = []
j = 0
for _ in range(20):
    temp_base = int((_ - (_ % 2))/2)
    drone_temp = drone(drone_id=_, home_drone= get_drone_base_by_id(temp_base), target_field=None, max_time_flight=480, total_time_target = None, current_location= get_drone_base_by_id(temp_base).position, list_tickets = [])
    drones_list.append(drone_temp)

In [373]:
#Printing generated drones
print('************ Generating Drones ************\n')

for _ in drones_list:
    print(_.drone_id, _.home_drone.base_id, _.target_field, _.max_time_flight, _.current_location, _.status)

************ Generating Drones ************

0 0 None 480 (16.5, 121.0667) AVAILABLE
1 0 None 480 (16.5, 121.0667) AVAILABLE
2 1 None 480 (16.7043, 121.8306) AVAILABLE
3 1 None 480 (16.7043, 121.8306) AVAILABLE
4 2 None 480 (17.6127, 121.8016) AVAILABLE
5 2 None 480 (17.6127, 121.8016) AVAILABLE
6 3 None 480 (18.088, 120.6098) AVAILABLE
7 3 None 480 (18.088, 120.6098) AVAILABLE
8 4 None 480 (16.7033, 121.6778) AVAILABLE
9 4 None 480 (16.7033, 121.6778) AVAILABLE
10 5 None 480 (18.2626, 121.9327) AVAILABLE
11 5 None 480 (18.2626, 121.9327) AVAILABLE
12 6 None 480 (17.6103, 120.6759) AVAILABLE
13 6 None 480 (17.6103, 120.6759) AVAILABLE
14 7 None 480 (16.972, 121.7091) AVAILABLE
15 7 None 480 (16.972, 121.7091) AVAILABLE
16 8 None 480 (18.3538, 121.4435) AVAILABLE
17 8 None 480 (18.3538, 121.4435) AVAILABLE
18 9 None 480 (17.1148, 121.7607) AVAILABLE
19 9 None 480 (17.1148, 121.7607) AVAILABLE


In [374]:
drone_distances = df_GeoDistances_anas.loc[f"drone_{drone.drone_id}"].sort_values()

print(drone_distances)

AttributeError: type object 'drone' has no attribute 'drone_id'

In [ ]:
def assign_targets_greedy(drones_list, tickets, df_GeoDistances, max_targets_per_drone=5):
    assigned_targets = set()

    # Filter available drones
    available_drones = [d for d in drones_list if d.status.lower() == 'available']

    for drone in available_drones:
        drone_distances = df_GeoDistances.loc[f"drone_{drone.drone_id}"].sort_values()

        assigned = 0
        for target_id in drone_distances.index:
            if target_id in assigned_targets:
                continue

            # Assign this target
            target = tickets[target_id]
            drone.assigned_targets.append(target_id)
            target.assign_drone(drone)
            target.update_status('in_progress')
            assigned_targets.add(target_id)
            assigned += 1

            if assigned >= max_targets_per_drone:
                break

        # Update drone status
        if assigned >= max_targets_per_drone:
            drone.status = 'BUSY'

    print(f"Assigned {len(assigned_targets)} targets.")


In [ ]:
assigned_targets = assign_targets_greedy(drones_list, tickets, df_GeoDistances_anas)

TypeError: list indices must be integers or slices, not str

In [ ]:
df_GeoDistances_anas = df_GeoDistances
targets = []
for i in range(100):
    for j in range(20):
        min_values = df_GeoDistances_anas.min(axis=0)
        targets.append(min_values)
        # print(min_values)
print(targets[0])

target_0     73.192931
target_1     71.630030
target_2     71.447638
target_3     73.197100
target_4     72.516138
               ...    
target_95    29.581857
target_96    28.659324
target_97    29.026301
target_98    29.248382
target_99    30.282729
Length: 100, dtype: float64


In [ ]:
print(df_GeoDistances_anas.iloc[0][1])

280.46480431327694


/var/folders/bj/7jjtf93x36z4c67ps065y0kc0000gn/T/ipykernel_22316/2942000010.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df_GeoDistances_anas.iloc[0][1])


In [ ]:
for i in range(100):
    for j in range(20):
        if targets[i][i] == df_GeoDistances_anas.iloc[i][j]:
            drone = get_drone_base_by_id(i)
            drone.status = 'PENDING'

/var/folders/bj/7jjtf93x36z4c67ps065y0kc0000gn/T/ipykernel_22316/2604262166.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if targets[i][i] == df_GeoDistances_anas.iloc[i][j]:


AttributeError: 'NoneType' object has no attribute 'status'

### mohammed

In [ ]:
df_GeoDistances = pd.DataFrame({
    f"target_{j}": [
        GeoCalculator.calculate_distance(*drone.current_location, *target.target_field)
        for drone in drones_list
    ]
    for j, target in enumerate(tickets)
}, index=[f"drone_{i}" for i in range(len(drones_list))])

In [ ]:
df_GeoDistances

,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,...,target_90,target_91,target_92,target_93,target_94,target_95,target_96,target_97,target_98,target_99
drone_0,282.445360,280.464804,280.346000,282.426592,281.423910,280.505651,282.851155,282.461475,280.496102,282.844723,...,46.061296,46.799373,196.639108,188.383867,188.303485,188.277335,189.227303,188.947729,188.816211,187.270486
drone_1,282.445360,280.464804,280.346000,282.426592,281.423910,280.505651,282.851155,282.461475,280.496102,282.844723,...,46.061296,46.799373,196.639108,188.383867,188.303485,188.277335,189.227303,188.947729,188.816211,187.270486
drone_2,246.316139,244.616522,244.448998,246.314475,245.537412,244.638223,246.830531,246.345160,244.628178,246.820753,...,46.862823,46.331778,151.620349,144.546650,144.465425,144.179609,145.072046,144.688041,144.447407,143.632155
drone_3,246.316139,244.616522,244.448998,246.314475,245.537412,244.638223,246.830531,246.345160,244.628178,246.820753,...,46.862823,46.331778,151.620349,144.546650,144.465425,144.179609,145.072046,144.688041,144.447407,143.632155
drone_4,145.488210,143.747489,143.584804,145.484403,144.676747,143.771463,145.991529,145.515752,143.761431,145.982115,...,103.274104,102.634286,52.049700,44.309726,44.226946,44.034556,44.970917,44.620402,44.419457,43.301698
drone_5,145.488210,143.747489,143.584804,145.484403,144.676747,143.771463,145.991529,145.515752,143.761431,145.982115,...,103.274104,102.634286,52.049700,44.309726,44.226946,44.034556,44.970917,44.620402,44.419457,43.301698
drone_6,162.747547,160.764291,160.785658,162.697884,161.514638,160.833575,162.800971,162.731312,160.827989,162.803576,...,168.730013,168.658252,139.580276,135.145681,135.132157,135.684939,135.994741,136.164149,136.373118,134.512043
drone_7,162.747547,160.764291,160.785658,162.697884,161.514638,160.833575,162.800971,162.731312,160.827989,162.803576,...,168.730013,168.658252,139.580276,135.145681,135.132157,135.684939,135.994741,136.164149,136.373118,134.512043
drone_8,247.302331,245.525474,245.367849,247.296729,246.462089,245.551729,247.795644,247.328606,245.541722,247.786560,...,30.909907,30.426426,153.712007,146.205229,146.122699,145.899189,146.823265,146.459385,146.242782,145.221703
drone_9,247.302331,245.525474,245.367849,247.296729,246.462089,245.551729,247.795644,247.328606,245.541722,247.786560,...,30.909907,30.426426,153.712007,146.205229,146.122699,145.899189,146.823265,146.459385,146.242782,145.221703


In [ ]:
def estimate_total_time(drone, distance_km, loiter_time=5):
    return 2 * ((distance_km/drone.speed)*60) + loiter_time  # round trip + loiter

In [ ]:
drone.__dict__

{'drone_id': 12,
 'drone_model': 'JOUVAC CW-30E',
 'home_drone': <__main__.drone_base at 0x11e1166a0>,
 'target_field': None,
 'max_time_flight': 480,
 'total_time_target': None,
 'current_location': (17.6103, 120.6759),
 'list_tickets': ['target_98', 'target_99'],
 'status': 'AVAILABLE',
 'speed': 90,
 'available_time_flight': np.float64(109.12129436670415)}

## Greedy Assignement

In [377]:
assigned_tickets = set()

for ticket_idx in df_GeoDistances.columns:
    # Get distances to this ticket from all drones
    distances = df_GeoDistances[ticket_idx]
    
    # Sort drones by distance to the ticket
    sorted_drones = distances.sort_values().index  # drone indices
    
    for drone_id in sorted_drones:
        drone = drones_list[int(drone_id.split('_')[1])]  # assuming index like 'drone_3'
        distance = distances[drone_id]
        total_time = estimate_total_time(drone,distance) # we assume that the mission is always examine and the loiter time is fixed 10min
        
        if total_time <= drone.available_time_flight:
            # Assign the ticket
            drone.list_tickets.append(ticket_idx)
            drone.available_time_flight -= total_time
            assigned_tickets.add(ticket_idx)
            break  # Go to next ticket


In [378]:
print("Assignment complete.")
for i, drone in enumerate(drones_list):
    print(f"Drone {i}: assigned tickets = {drone.list_tickets}, time left = {drone.available_time_flight:.1f} min")

print(f"Total tickets assigned: {len(assigned_tickets)} / {df_GeoDistances.shape[1]}")


Assignment complete.
Drone 0: assigned tickets = ['target_81', 'target_82'], time left = 362.3 min
Drone 1: assigned tickets = [], time left = 480.0 min
Drone 2: assigned tickets = ['target_65', 'target_66', 'target_67', 'target_68', 'target_69', 'target_70', 'target_71', 'target_72', 'target_73', 'target_74'], time left = 11.4 min
Drone 3: assigned tickets = ['target_75', 'target_76', 'target_77', 'target_78', 'target_79'], time left = 249.1 min
Drone 4: assigned tickets = ['target_16', 'target_17', 'target_20'], time left = 21.7 min
Drone 5: assigned tickets = ['target_26', 'target_27', 'target_28', 'target_29', 'target_30', 'target_33', 'target_34', 'target_35'], time left = 16.7 min
Drone 6: assigned tickets = ['target_96', 'target_97'], time left = 107.1 min
Drone 7: assigned tickets = [], time left = 480.0 min
Drone 8: assigned tickets = ['target_48', 'target_49', 'target_50', 'target_51', 'target_52', 'target_53', 'target_54', 'target_55', 'target_56', 'target_57', 'target_58', 

## Visualizing Map

In [ ]:
import folium
from collections import defaultdict

def slightly_offset(lat, lon, index, offset=0.0002):
    """Offsets a coordinate slightly based on index."""
    return lat + (index * offset), lon + (index * offset)

# Count occurrences of each position
position_counts = defaultdict(int)

# Center map
all_positions = [drone.current_location for drone in drones_list] + \
                [ticket.target_field for ticket in tickets]
avg_lat = sum(lat for lat, _ in all_positions) / len(all_positions)
avg_lon = sum(lon for _, lon in all_positions) / len(all_positions)

m = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

# Add drone markers (blue)
for i, drone in enumerate(drones_list):
    pos = drone.current_location
    position_counts[pos] += 1
    lat, lon = slightly_offset(*pos, position_counts[pos])
    folium.Marker(
        location=(lat, lon),
        popup=f"Drone {i}",
        icon=folium.Icon(color='blue', icon='plane', prefix='fa')
    ).add_to(m)

# Add ticket markers (red)
for j, ticket in enumerate(tickets):
    pos = ticket.target_field
    position_counts[pos] += 1
    lat, lon = slightly_offset(*pos, position_counts[pos])
    folium.Marker(
        location=(lat, lon),
        popup=f"Ticket {j}",
        icon=folium.Icon(color='red', icon='flag', prefix='fa')
    ).add_to(m)

m.save("geo_map_new.html")


## GeoCalculator Class

In [ ]:
import math

class GeoCalculator:
    EARTH_RADIUS_KM = 6371.0

    @staticmethod
    def calculate_distance(lat1, lon1, lat2, lon2):
        lat1_rad = GeoCalculator.degrees_to_radians(lat1)
        lon1_rad = GeoCalculator.degrees_to_radians(lon1)
        lat2_rad = GeoCalculator.degrees_to_radians(lat2)
        lon2_rad = GeoCalculator.degrees_to_radians(lon2)

        delta_lat = lat2_rad - lat1_rad
        delta_lon = lon2_rad - lon1_rad

        a = (math.sin(delta_lat / 2) ** 2 +
             math.cos(lat1_rad) * math.cos(lat2_rad) *
             math.sin(delta_lon / 2) ** 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return GeoCalculator.EARTH_RADIUS_KM * c

    @staticmethod
    def calculate_bearing(lat1, lon1, lat2, lon2):
        lat1_rad = GeoCalculator.degrees_to_radians(lat1)
        lon1_rad = GeoCalculator.degrees_to_radians(lon1)
        lat2_rad = GeoCalculator.degrees_to_radians(lat2)
        lon2_rad = GeoCalculator.degrees_to_radians(lon2)

        delta_lon = lon2_rad - lon1_rad

        y = math.sin(delta_lon) * math.cos(lat2_rad)
        x = (math.cos(lat1_rad) * math.sin(lat2_rad) -
             math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(delta_lon))
        bearing_rad = math.atan2(y, x)
        return (GeoCalculator.radians_to_degrees(bearing_rad) + 360) % 360

    @staticmethod
    def degrees_to_radians(degrees):
        return degrees * math.pi / 180.0

    @staticmethod
    def radians_to_degrees(radians):
        return radians * 180.0 / math.pi


In [ ]:
home_lat = 48.8566         # Paris
home_lon = 2.3522
destination_lat = 51.5074  # London
destination_lon = -0.1278

progress = 0.0005  # Small step along the line
new_lat = home_lat + (destination_lat - home_lat) * progress
new_lon = home_lon + (destination_lon - home_lon) * progress

print(f"Interpolated point: Latitude = {new_lat}, Longitude = {new_lon}")

Interpolated point: Latitude = 48.8579254, Longitude = 2.3509599999999997
